In [1]:
# Aurthor: Elihu Essien-Thomspon
# Program Description: Experiment 1
# Finding the best Genetic Algorithm
# Representative to use for the final
# Experiment.

import numpy as np
import random
import math

In [3]:
# Returns the distance between two 
# points(x, y) on the map

def cityDistance(map, cityID1, cityID2):
    x = abs(map[cityID1][0] - map[cityID2][0])
    y = abs(map[cityID1][1] - map[cityID2][1])
    
    # pythagoras
    dist = math.sqrt(x**2 + y**2)
    return(dist)

In [ ]:
# Reading in file contaning city vectors
# to construct a distance matrix soring
# the lenght of each edge in the map
def getMap(size,id):
    cities = np.empty((0,2), dtype=int)      # initialize empty 2D vector list
    
    # open file
    with open(f"C:/Users/C14460702/Dissertation/Data/Maps/Size - {size}/map{id}.txt", 'r') as f:
        # extract vector list data from the comma separated string list data
        for line in f:
            line = line.strip('\n')    # remove '\n'
            vec = line.split(', ')     # split by ', '
            
            # cast co-ordinate vector as integers and insert into list 
            cities = np.append(cities, [[ int(vec[0]), int(vec[1]) ]], axis=0)
    
    # clost file
    f.close()
    
    
    # initialize distance table
    distanceTable = np.zeros((mapSize,mapSize))
    for row in range(len(cities)):
        for col in range(len(cities)):
            distanceTable[row][col] = cityDistance(cities, row, col)
    
    # return distance matrix
    return (distanceTable)

In [4]:
# Function takes in a route list
# and a distnce matrix to be used
# to score the route given

def routeScore(route, distanceTable):
    
    dist = 0;
    
    # aggregate distances between cities
    # on the list given
    for i in range(len(route)):
        cur = route[i]
        nxt = route[(i+1) % len(route)]  # includes loop back to start
        dist += distanceTable[cur][nxt]  # get the distances from the matrix
        
    
    # the smaller the distance, the higher the score
    return (1/dist)
    

In [5]:
# maps a variable exising on one range to another range
def mapRanges(value, currlow, currHigh, newLow, newHigh):
    # extract ranges
    currRange = currHigh - currlow
    newRange = newHigh - newLow
    
    # percentage of range that the value takes up
    valPercent = float(value - currlow) / float(currRange)
    
    # translate percentage to new range
    newVal = (newRange * valPercent) + newLow
    
    return(newVal)

In [6]:
# This fitness function simultaneously picks out {popSize} number of mating candidates
# from a wheel of potential candidates. The fitness of each candidate is calculated 
# by the scores passed in from the 'population evaluation' step (individuals scoring
# similar results would earn a similar number of slots) and it is represented by the
# proportion of space they take up on a matting Pool selection wheel.

def StochasticUniversalSampling(population, scores, popSize, mapSize):
    
    # evaluate what percentage of the wheel each individual
    # should recieve based on thier scores
    total = sum(scores)
        
    for i in range(popSize):     # evaluated percentage for individual scores
        scores[i] = scores[i]/total
    
    # If there were only 10 members of the population, the score percentages would 
    # be within reasonable ranges. On average, each member would have approximately
    # 10% of the pie, which could then logically equate to approx. 10 slots on a 
    # 100 slot wheel. However, as the population increases to 100 or more members,
    # the average % may decrease to 0.1 or less which is then difficult to assign 
    # slots for. So to combat this possiblility and account for code flexibility, 
    # I decided to further normalize these percentages before calculating the fitness 
    # values (number of slots they earn on the wheel).
    
    Max = max(scores)
    Min = min(scores)
    
    Fitness_Scores = []
    for i in range(popSize):
        # I picked 10 as the max amount of slots a member can have
        fitnessValue = mapRanges(scores[i], Min, Max, 0, 10)
        remainder = fitnessValue % 1
        
        # finally, scaling with the decimal parts, each individual gets
        # a {remainder}% chance to add an extra slot to the wheel.
        # The individual scoring last is awarded only 1 slot on the wheel.
        if(fitnessValue > 1):
            Fitness_Scores += [math.ceil(fitnessValue)] if (random.random() < remainder) else [math.floor(fitnessValue)]
        else:
            Fitness_Scores += [1]
    
    
    # Create roulette wheel type mating pool with fitness coresponding to the slots alloted
    # I used a numpy array to increase algorithm speed
    wheel = np.empty((0,mapSize), dtype=int)
    
    # the number of slots alloted on the wheel would be 
    # between [0-10] and is calculated by (10 - [Normalized Fitness]) 
    for i in range(popSize):
        for j in range(Fitness_Scores[i]):   # add {fitness} amount of coppies for each member to the wheel
            wheel = np.append(wheel,[population[i]],axis=0)
    
    
    # interval used to simultaneously select {popSize} mating indivduals
    wheelSize = np.size(wheel,axis=0)
    interval = int(wheelSize/popSize)
    
    # spin wheel
    np.random.shuffle(wheel)
    
    matingPool = []                                 # empty mating pool
    for i in range(0, wheelSize, interval):         # evenly spaced pointer locations
        matingPool += [wheel[i].tolist()]                    # populate mating pool based on pointers
        
    return(matingPool)

In [7]:
# This fitness function simultaneously picks out {popSize} number of mating 
# candidates from a wheel of potential candidates. The candidates are ranked 
# based on thier evaluated scores and that rank becomes thier fitness score, 
# represented by the proportion of space they take up on a matting Pool
# selection wheel.

def RankBasedSampling(population, scores, popSize, mapSize):
    
    # sort population by score accending
    sortedPopulation = [i for _,i in sorted(zip(scores,population))]
    
    # Create roulette wheel type mating pool with fitness coresponding to the slots alloted
    wheel = np.empty((0, mapSize), dtype=int)
    
    for fitness in range(popSize):        # fitness coresponds to location in sorted the array (ID)
        for i in range(fitness+1):        # e.g. ID[0] gets 1 slot and ID[last] gets {popSize} slots
            wheel = np.append(wheel,[sortedPopulation[fitness]],axis=0)
            
    
    # interval used to simultaneously select {popSize} mating indivduals
    wheelSize = np.size(wheel,axis=0)
    interval = int(wheelSize/popSize)
    
    # spin wheel
    np.random.shuffle(wheel)
    
    matingPool = []                                 # empty mating pool
    for i in range(0, wheelSize, interval):         # evenly spaced pointer locations
        matingPool += [wheel[i].tolist()]           # populate mating pool based on pointers
        
    return(matingPool)

In [8]:
# Rather than matting Pool selection wheel, a tournament is held between
# pairs selected from the population and depeding on the tournament
# probablility, either the winner or loser of the tournament continues
# on to the mating pool for the next generation. The tournament winner 
# is determinged by the recieved evaluation scores.

def TournamentSampling (population, scores, popSize, mapSize):
    
    matingPool = []         # empty mating pool array
    tournamentProb = 0.75   # probabilty chosing the winner is 75%
    
    for i in range(popSize):
        # choose 2 random members to compete
        ID1 = random.choice(range(popSize))
        ID2 = random.choice(range(popSize))
        score1 = scores[ID1]
        score2 = scores[ID2]
        
        winnerID = 0   # tournament winner placholder ID
        
        # depending on the probability, we will chose either the higher or lower
        # scoring member to add to the matting pool
        if(random.random() < tournamentProb):
            winnerID = ID1 if (score1>score2) else ID2  # the greatest wins
        else:
            winnerID = ID1 if (score1<score2) else ID2  # the lowest wins
        
        # populate mating pool
        matingPool += [population[winnerID]]
    
    return(matingPool)

In [9]:
# Steady State Function takes in 4 routes
# and returns for mating only the 2 children
# scoring greatest 

# The algorithm works by splitting the 4 routes
# into 2 groups (a-b) and (c-d) and by comparing
# the winners of the groups, the best two can be
# isolated and sent back
def SteadyStateFunction(routeA,routeB,routeC,routeD, distanceTable):
    # scores
    a = routeScore(routeA,distanceTable)
    b = routeScore(routeB,distanceTable)
    c = routeScore(routeC,distanceTable)
    d = routeScore(routeD,distanceTable)
    
    
    if(a>b):                                    # a is the greatest from Group1
        if(c>d):                                  # c is the greatest from Group2
            if(d>a):                                # if the greatest from Group1 is lower than the lowest from Group2
                return(routeC,routeD)                   # Group2 wins
            else:                                   # else the greatest in group1 has won twice so it's definately being chosen
                if(b>c):                              # is the lowest in Group1 also greater than the greatest in Group2
                    return(routeA,routeB)                 # if so, Group1 wins
                else:
                    return(routeA,routeC)                 # if not, only the greatest in Group1 and in Group2 win
        else:                                     # Same logic as above but the greatest of Group2 happened to be d not c
            if(c>a):                                # ......
                return(routeC,routeD)               # ......
            else: 
                if(b>d):
                    return(routeA,routeB)
                else:
                    return(routeA,routeD)
    else:                                       # Same logic as above but the greatest of Group1 happened to be b not a
        if(c>d):                                  # ......
            if(d>b):                              # ......
                return(routeC,routeD) 
            else: 
                if(a>c): 
                    return(routeA,routeB) 
                else:
                    return(routeB,routeC) 
        else:                                     # Same logic as above but the greatest of Group2 happened to be d not c
            if(c>b):                                # ......
                return(routeC,routeD)               # ......
            else:
                if(a>d):
                    return(routeA,routeB)
                else:
                    return(routeB,routeD)

In [10]:
def GA(distanceTable, mapSize, FitnessFunction, popSize, maxIterations, mutationProb, elitePercentage = 0, useSteadyState = False):
    # elitism is defaulted to false and the percentage is defaulted to the top 1/4 of the population
    
    # generate cityIDs list representing the 
    # default chromosome or individual
    defaultGene = [i for i in range(mapSize)]
    
    # 2D list to store the chromosomes
    population = []
    
    # Algorithm record
    GBestScore_PerItreation = []
    
    
    # Step 1 - Create initial population
    # populate list with initial random individuals
    random.seed(random.random()) # change the seed each algorithm run
    for i in range(popSize):
        chromosome = random.sample(defaultGene, mapSize)
        population += [chromosome]
    
     # ---begin algorithm loop--- 
    for i in range(maxIterations):
        
        # Step 2 - Evaluatuation
        # score all members of the population
        scores = []
        bestScore = 0
        for i in range(popSize):
            score = routeScore(population[i], distanceTable)
            scores += [score]
            
            if(score > bestScore):
                bestScore = score
                
        GBestScore_PerItreation += [bestScore]
        
        # Step 3 - Create Mating Pool using specified Fitness function
        matingPool = FitnessFunction(population, scores, popSize, mapSize)
        
        
        # Step 4 - Breeding (with option for using Elitism)
        # eliteSize defaults to 0 if not using elitism
        eliteSize = round(popSize * elitePercentage)
        
        # if using elitism, skip the first {eliteSize} members of the population
        # before breeding and replacing the later members.
        for i in range(eliteSize, popSize, 2):
            parent1 = matingPool[i]
            parent2 = matingPool[(i+1) % (popSize-1)]  # % operator incase of out-of-bounds error
            
            # Pick a random swap point (not at the ends)
            swapPoint = random.choice(range(1, mapSize-1))
            
            # initialize 2 empty children
            child1 = [0] * swapPoint
            child2 = [0] * swapPoint

            # fill first portion with initial parent genes 
            for j in range(swapPoint):
                child1[j] = parent1[j]
                child2[j] = parent2[j]

            # fill second portion with partner parent genes
            child1 = child1 + [gene for gene in parent2 if gene not in child1]
            child2 = child2 + [gene for gene in parent1 if gene not in child2]
            
            
            
            # Step 5 - Mutation 
            # performed with the mutation probability
            # for each gene in the children
            for gene in range(mapSize):
                
                # child1 swap check
                if(random.random() < mutationProb):
                    # pick a random partner to swap with
                    # making sure it parnter is not self
                    gene2 = gene
                    while (gene2 == gene):
                        gene2 = random.choice(range(mapSize))
                    
                    # swap genes
                    temp = child1[gene]
                    child1[gene] = child1[gene2]
                    child1[gene2] = temp


                # child2 swap check
                if(random.random() < mutationProb):
                    # pick a random partner to swap with
                    # making sure it parnter is not self
                    gene2 = gene
                    while (gene2 == gene):
                        gene2 = random.choice(range(mapSize))
                    
                    # swap genes
                    temp = child2[gene]
                    child2[gene] = child2[gene2]
                    child2[gene2] = temp
            
            
            # Step 6 - Replacement of old with new
            # steady state reproduction aims to only keep the best results
            if (useSteadyState):
                population[i], population[(i+1) % (popSize-1)] = SteadyStateFunction(parent1, parent2, child1, child2, distanceTable)
            else:
                # simply replace previous generation with next generation
                population[i] = child1
                population[(i+1) % (popSize-1)] = child2
    
    return(GBestScore_PerItreation)

In [11]:
# Genetic algorithm settings
popSize = 50            # Number of chromosomes to use
maxItterations = 300    # Maximum number of generations
mutationProb = 0.006    # Mutation Probability of 0.6%

# Traveling Salesman Problem Settings
mapSize = 10        # cities per map
mapCounter = 0      # itteration counter

In [11]:
# Expriment 1A: Which of the GA fitness function variants 
# performs the best?
#
# Performance is measured primarily by speed of convergence, 
# however, a 'too high' level of convergence pressure can force
# the algorithm to settle on a 'local' optimum rather than 'global'
# so a balance must be set between convergence speed and final value.
# With this in mind, I record both the score of the final solution
# converged on and the itteration number that that solution was
# found.

# Experiment Settings
mapCounter = 0      # itteration counter

# average values for each itteration
SUS_results_avg = [0] * maxItterations
RBS_results_avg = [0] * maxItterations
TS_results_avg  = [0] * maxItterations


f1 = open(f"C:/Users/C14460702/Dissertation/Data/Results/Experiment1A/Size - {mapSize}/StochasticUniversalSampling_data.txt", 'w')
f2 = open(f"C:/Users/C14460702/Dissertation/Data/Results/Experiment1A/Size - {mapSize}/RankingBasedSampling_data.txt", 'w')
f3 = open(f"C:/Users/C14460702/Dissertation/Data/Results/Experiment1A/Size - {mapSize}/TournamentSampling_data.txt", 'w')
while (True):
    try:
        distanceTable = getMap(mapSize,mapCounter)
        
        results1 = GA(distanceTable, mapSize, StochasticUniversalSampling, popSize, maxItterations, mutationProb)
        results2 = GA(distanceTable, mapSize, RankBasedSampling, popSize, maxItterations, mutationProb)
        results3 = GA(distanceTable, mapSize, TournamentSampling, popSize, maxItterations, mutationProb)
        
        f1.writelines(str(results1))
        f2.writelines(str(results2))
        f3.writelines(str(results3))
        
        print(f"Finished map {mapCounter+1}!")
        mapCounter += 1     # update counter
    except FileNotFoundError:
        break



f1.close()
f2.close()
f3.close()

Finished map 1!
Finished map 2!
Finished map 3!
Finished map 4!
Finished map 5!
Finished map 6!
Finished map 7!
Finished map 8!
Finished map 9!
Finished map 10!
Finished map 11!
Finished map 12!
Finished map 13!
Finished map 14!
Finished map 15!
Finished map 16!
Finished map 17!
Finished map 18!
Finished map 19!
Finished map 20!
Finished map 21!
Finished map 22!
Finished map 23!
Finished map 24!
Finished map 25!
Finished map 26!
Finished map 27!
Finished map 28!
Finished map 29!
Finished map 30!
Finished map 31!
Finished map 32!
Finished map 33!
Finished map 34!
Finished map 35!
Finished map 36!
Finished map 37!
Finished map 38!
Finished map 39!
Finished map 40!
Finished map 41!
Finished map 42!
Finished map 43!
Finished map 44!
Finished map 45!
Finished map 46!
Finished map 47!
Finished map 48!
Finished map 49!
Finished map 50!
Finished map 51!
Finished map 52!
Finished map 53!
Finished map 54!
Finished map 55!
Finished map 56!
Finished map 57!
Finished map 58!
Finished map 59!
Finish

In [13]:
# Expriment 1B: Does ELitism bring any aditional value?
#
# from the results of experiment 1A, the Stochastic
# Universal Sampling fitness texnique was chosen as
# the best representative for the GA to be used as a
# base line for this experiment.

# Experiment Settings
mapCounter = 0      # itteration counter

# Elitist settings
startPerc = 0
endPerc = 100
increment = 10



# DataSets
# dynamically create and open file variables 
for i in range(startPerc,endPerc,increment):
    globals()[f"f{int(i/increment)}"] = open(f"C:/Users/C14460702/Dissertation/Data/Results/Experiment1B/Size - {mapSize}/{i}%.txt", 'w')

while (True):
    try:
        distanceTable = getMap(mapSize,mapCounter)
                
        for i in range(startPerc,endPerc,increment):
            data = GA(distanceTable, mapSize, RankBasedSampling, popSize, maxItterations, mutationProb, i/100)
            globals()[f"f{int(i/increment)}"].writelines(str(data))
        
        
        print(f"Finished map {mapCounter+1}!")
        mapCounter += 1     # update counter
    except FileNotFoundError:
        break
        
for i in range(startPerc,endPerc,increment):
    globals()[f"f{int(i/increment)}"].close()

Finished map 1!
Finished map 2!
Finished map 3!
Finished map 4!
Finished map 5!
Finished map 6!
Finished map 7!
Finished map 8!
Finished map 9!
Finished map 10!
Finished map 11!
Finished map 12!
Finished map 13!
Finished map 14!
Finished map 15!
Finished map 16!
Finished map 17!
Finished map 18!
Finished map 19!
Finished map 20!
Finished map 21!
Finished map 22!
Finished map 23!
Finished map 24!
Finished map 25!
Finished map 26!
Finished map 27!
Finished map 28!
Finished map 29!
Finished map 30!
Finished map 31!
Finished map 32!
Finished map 33!
Finished map 34!
Finished map 35!
Finished map 36!
Finished map 37!
Finished map 38!
Finished map 39!
Finished map 40!
Finished map 41!
Finished map 42!
Finished map 43!
Finished map 44!
Finished map 45!
Finished map 46!
Finished map 47!
Finished map 48!
Finished map 49!
Finished map 50!
Finished map 51!
Finished map 52!
Finished map 53!
Finished map 54!
Finished map 55!
Finished map 56!
Finished map 57!
Finished map 58!
Finished map 59!
Finish

In [12]:
# Experiment 1C - Considering Steady-State technique
# from the previous experiment, 20% was chosen as the
# best elite percencentage to use for the algorithm.
# In this experiment, we will compare 3 versions:
# -Base Algorithm + Steady State
# -vs
# -Elitist Algorithm
# -vs
# -Elitist Algorithm + Steady State


# Experiment Settings
mapCounter = 0      # itteration counter
OptimumElitePercent = 0.1;


# Experiment 1C data files
f1 = open(f"C:/Users/C14460702/Dissertation/Data/Results/Experiment1C/Size - {mapSize}/Base_SS.txt", 'w')
f2 = open(f"C:/Users/C14460702/Dissertation/Data/Results/Experiment1C/Size - {mapSize}/Elt.txt", 'w')
f3 = open(f"C:/Users/C14460702/Dissertation/Data/Results/Experiment1C/Size - {mapSize}/Elt_SS.txt", 'w')

while (True):
    try:
        distanceTable = getMap(mapSize,mapCounter)
        
        
                
        base_SS = GA(distanceTable, mapSize, RankBasedSampling, popSize, maxItterations, mutationProb, 0, True)
        Elt = GA(distanceTable, mapSize, RankBasedSampling, popSize, maxItterations, mutationProb, OptimumElitePercent)
        Elt_SS = GA(distanceTable, mapSize, RankBasedSampling, popSize, maxItterations, mutationProb, OptimumElitePercent, True)
            
        f1.writelines(str(base_SS))
        f2.writelines(str(Elt))
        f3.writelines(str(Elt_SS))
        
        print(f"Finished map {mapCounter+1}!")
        mapCounter += 1     # update counter
    except FileNotFoundError:
        break


# close files 
f1.close()
f2.close()
f3.close()

Finished map 1!
Finished map 2!
Finished map 3!
Finished map 4!
Finished map 5!
Finished map 6!
Finished map 7!
Finished map 8!
Finished map 9!
Finished map 10!
Finished map 11!
Finished map 12!
Finished map 13!
Finished map 14!
Finished map 15!
Finished map 16!
Finished map 17!
Finished map 18!
Finished map 19!
Finished map 20!
Finished map 21!
Finished map 22!
Finished map 23!
Finished map 24!
Finished map 25!
Finished map 26!
Finished map 27!
Finished map 28!
Finished map 29!
Finished map 30!
Finished map 31!
Finished map 32!
Finished map 33!
Finished map 34!
Finished map 35!
Finished map 36!
Finished map 37!
Finished map 38!
Finished map 39!
Finished map 40!
Finished map 41!
Finished map 42!
Finished map 43!
Finished map 44!
Finished map 45!
Finished map 46!
Finished map 47!
Finished map 48!
Finished map 49!
Finished map 50!
Finished map 51!
Finished map 52!
Finished map 53!
Finished map 54!
Finished map 55!
Finished map 56!
Finished map 57!
Finished map 58!
Finished map 59!
Finish